<a href="https://colab.research.google.com/github/Sneha-mav/ev-zone-mapping-routing-delhi/blob/main/Clusterbasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = '/content/drive/MyDrive/EV Project Davise Lab/ev_final.xlsx'
df = pd.read_excel(file_path)

In [ ]:
df.head(3)

,uid,name,vendor_name,address,latitude,longitude,city,country,open,close,...,postal_code,zone,0,available,capacity,cost_per_unit,power_type,total,type,vehicle_type
0,STATIC12,GensolCharge Pvt. Ltd.,GensolCharge Pvt. Ltd.,"NDSE Grid, BRPL South Extension",28.568238,77.219666,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,15 kW,NaN,DC,2.0,BEVC DC 001,['4W']
1,STATIC14,REIL,REIL,Scada office kalka ji,28.541995,77.260583,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,3.3 kW,NaN,AC,3.0,BEVC AC 001,"['2W', '3W', '4W']"
2,STATIC15,REIL,REIL,Ashram Chowk Mathura Road,28.571189,77.259806,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,15 kW,NaN,DC,2.0,BEVC DC 001,['4W']


In [ ]:
from math import radians, cos, sin, asin, sqrt
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [ ]:
# Function to calculate haversine distance between two coordinates (in km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

In [ ]:
def find_nearest_station_and_zone(lat, lon, df, top_n=3):
    # Bounding box check
    if not (df['latitude'].min() <= lat <= df['latitude'].max() and
            df['longitude'].min() <= lon <= df['longitude'].max()):
        return {'in_range': False, 'message': 'Coordinates out of dataset range.'}

    # Compute haversine distance
    df['distance_km'] = df.apply(lambda row: haversine(lat, lon, row['latitude'], row['longitude']), axis=1)

    # Handle cases where vehicle_type is a string like "['2W', '4W']"
    def clean_vehicle_type(val):
        if isinstance(val, str):
            try:
                return eval(val)
            except:
                return []
        elif isinstance(val, list):
            return val
        else:
            return []

    df['vehicle_type'] = df['vehicle_type'].apply(clean_vehicle_type)

    # Group by unique locations and aggregate vehicle types
    df_grouped = df.groupby(['name', 'address', 'latitude', 'longitude', 'zone']).agg({
        'distance_km': 'min',
        'vehicle_type': lambda x: sorted(set(i for sublist in x for i in sublist))
    }).reset_index()

    # Sort by distance and select top N
    nearest_stations = df_grouped.sort_values('distance_km').head(top_n)

    # Nearest zone
    nearest_zone = nearest_stations.iloc[0]['zone']

    # Format result
    result = {
        'in_range': True,
        'zone': nearest_zone,
        'nearest_stations': nearest_stations[['name', 'address', 'zone', 'distance_km', 'vehicle_type']].to_dict(orient='records')
    }
    return result


In [ ]:
# test case
vehicle_lat = 28.561
vehicle_lon = 77.22

result = find_nearest_station_and_zone(vehicle_lat, vehicle_lon, df, top_n=3)
print(result)


{'in_range': True, 'zone': 'central-delhi', 'nearest_stations': [{'name': 'GensolCharge Pvt. Ltd.', 'address': 'NDSE Grid, BRPL South Extension', 'zone': 'central-delhi', 'distance_km': 0.8054895961027215, 'vehicle_type': ['4W']}, {'name': 'REVOS', 'address': '13485 yusuf sarai mkt new delhi 110016', 'zone': 'south-west-delhi', 'distance_km': 1.308856480730356, 'vehicle_type': ['2W', '3W']}, {'name': 'REVOS', 'address': '13485 yusuf sarai market', 'zone': 'south-west-delhi', 'distance_km': 1.3173564902277397, 'vehicle_type': ['2W', '3W']}]}


In [ ]:
def print_result(result):
    if not result['in_range']:
        print("🚫 The given coordinates are outside the range of known charging stations.")
        print("📍 Message:", result['message'])
        return

    print(f"\n✅ Vehicle is within range. Zone: **{result['zone']}**\n")
    print("🔌 Nearest Charging Stations:\n")

    for i, station in enumerate(result['nearest_stations'], start=1):
        print(f"📍 Station #{i}")
        print(f"   - Name       : {station['name']}")
        print(f"   - Address    : {station['address']}")
        print(f"   - Zone       : {station['zone']}")
        print(f"   - Distance   : {station['distance_km']:.2f} km")
        print(f"   - Vehicles   : {', '.join(station['vehicle_type'])}")
        print()


In [ ]:
print_result(result)


✅ Vehicle is within range. Zone: **central-delhi**

🔌 Nearest Charging Stations:

📍 Station #1
   - Name       : GensolCharge Pvt. Ltd.
   - Address    : NDSE Grid, BRPL South Extension
   - Zone       : central-delhi
   - Distance   : 0.81 km
   - Vehicles   : 4W

📍 Station #2
   - Name       : REVOS
   - Address    : 13485 yusuf sarai mkt new delhi 110016
   - Zone       : south-west-delhi
   - Distance   : 1.31 km
   - Vehicles   : 2W, 3W

📍 Station #3
   - Name       : REVOS
   - Address    : 13485 yusuf sarai market
   - Zone       : south-west-delhi
   - Distance   : 1.32 km
   - Vehicles   : 2W, 3W

